##### Copyright 2020 The TensorFlow Authors.

## Before we start
To edit the colab notebook, please go to "File" -> "Save a copy in Drive" and make any edits on your copy.

Before we start, please run the following to make sure that your environment is
correctly setup. If you don't see a greeting, please refer to the
[Installation](../install.md) guide for instructions. 

In [ ]:
#@title Upgrade tensorflow_federated and load TensorBoard
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()

%load_ext tensorboard

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [ ]:
#@title
import collections
from matplotlib import pyplot as plt
from IPython.display import display, HTML, IFrame

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

np.random.seed(0)

def greetings():
  display(HTML('<b><font size="6" color="#ff00f4">Greetings, virtual tutorial participants!</font></b>'))
  return True
l = tff.federated_computation(greetings)()

# TensorFlow Federated for Image Classification

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/federated/blob/v0.14.0/docs/tutorials/federated_learning_for_image_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/federated/blob/v0.14.0/docs/tutorials/federated_learning_for_image_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Let's experiment with federated learning in simulation. In this tutorial, we use the classic MNIST training example to introduce the
Federated Learning (FL) API layer of TFF, `tff.learning` - a set of
higher-level interfaces that can be used to perform common types of federated
learning tasks, such as federated training, against user-supplied models
implemented in TensorFlow.


# 튜토리얼 개요

We'll be training a model to perform image classification using the classic MNIST dataset, with the neural net learning to classify digit from image. In this case, we'll be simulating federated learning, with the training data distributed on different devices.

<p><b>섹션</b></p>

1. Load TFF Libraries.
2. Explore/preprocess federated EMNIST dataset.
3. 모델을 만듭니다.
4. Set up federated averaging process for training.
5. Analyze training metrics.
6. Set up federated evaluation computation.
7. Analyze evaluation metrics.


## Preparing the input data

Let's start with the data. Federated learning requires a federated data set,
i.e., a collection of data from multiple users. Federated data is typically
non-[i.i.d.](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables),
which poses a unique set of challenges. Users typically have different distributions of data depending on usage patterns.

In order to facilitate experimentation, we seeded the TFF repository with a few
datasets.

Here's how we can load our sample dataset.

In [ ]:
# Code for loading federated data from TFF repository
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

The data sets returned by `load_data()` are instances of
`tff.simulation.datasets.ClientData`, an interface that allows you to enumerate the set
of users, to construct a `tf.data.Dataset` that represents the data of a
particular user, and to query the structure of individual elements.

Let's explore the dataset.

In [ ]:
len(emnist_train.client_ids)

In [ ]:
# Let's look at the shape of our data
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])

example_dataset.element_spec

In [ ]:
# Let's select an example dataset from one of our simulated clients
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])

# Your code to get an example element from one client:
example_element = next(iter(example_dataset))

example_element['label'].numpy()

In [ ]:
plt.imshow(example_element['pixels'].numpy(), cmap='gray', aspect='equal')
plt.grid(False)
_ = plt.show()

**Exploring non-iid data**

In [ ]:
## Example MNIST digits for one client
f = plt.figure(figsize=(20,4))
j = 0

for e in example_dataset.take(40):
  plt.subplot(4, 10, j+1)
  plt.imshow(e['pixels'].numpy(), cmap='gray', aspect='equal')
  plt.axis('off')
  j += 1

In [ ]:
# Number of examples per layer for a sample of clients
f = plt.figure(figsize=(12,7))
f.suptitle("Label Counts for a Sample of Clients")
for i in range(6):
  ds = emnist_train.create_tf_dataset_for_client(emnist_train.client_ids[i])
  k = collections.defaultdict(list)
  for e in ds:
    k[e['label'].numpy()].append(e['label'].numpy())
  plt.subplot(2, 3, i+1)
  plt.title("Client {}".format(i))
  for j in range(10):
    plt.hist(k[j], density=False, bins=[0,1,2,3,4,5,6,7,8,9,10])


In [ ]:
# Let's play around with the emnist_train dataset.
# Let's explore the non-iid charateristic of the example data.

for i in range(5):
  ds = emnist_train.create_tf_dataset_for_client(emnist_train.client_ids[i])
  k = collections.defaultdict(list)
  for e in ds:
    k[e['label'].numpy()].append(e['pixels'].numpy())
  f = plt.figure(i, figsize=(12,5))
  f.suptitle("Client #{}'s Mean Image Per Label".format(i))
  for j in range(10):
    mn_img = np.mean(k[j],0)
    plt.subplot(2, 5, j+1)
    plt.imshow(mn_img.reshape((28,28)))#,cmap='gray') 
    plt.axis('off')

# Each client has different mean images -- each client will be nudging the model
# in their own directions.

### Preprocessing the data

Since the data is already a `tf.data.Dataset`,  preprocessing can be accomplished using Dataset transformations. [See here](https://www.tensorflow.org/guide/data) for more detail on these transformations.

In [ ]:
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER=10

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1, 784]),
        y=tf.reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

Let's verify this worked.

In [ ]:
preprocessed_example_dataset = preprocess(example_dataset)

sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_dataset)))

sample_batch

Here's a simple helper function that will construct a list of datasets from the
given set of users as an input to a round of training or evaluation.

In [ ]:
def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
  ]

Now, how do we choose clients?

In [ ]:
sample_clients = emnist_train.client_ids[0:NUM_CLIENTS]

# Your code to get the federated dataset here for the sampled clients:
federated_train_data = make_federated_data(emnist_train, sample_clients)

print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))

## Keras로 모델 만들기

If you are using Keras, you likely already have code that constructs a Keras model. Here's an example of a simple model that will suffice for our needs.

In [ ]:
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])

**Centralized training with Keras**

In [ ]:
## Centralized training with keras ---------------------------------------------

# This is separate from the TFF tutorial, and demonstrates how to train a
# Keras model in a centralized fashion (contrasting training in a federated env)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255

y_train = y_train.astype("float32")

mod = create_keras_model()
mod.compile(
    optimizer=tf.keras.optimizers.RMSprop(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
h = mod.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2
)

# ------------------------------------------------------------------------------

**Federated training using a Keras model**

In order to use any model with TFF, it needs to be wrapped in an instance of the
`tff.learning.Model` interface.

More keras metrics you can add are [found here](https://www.tensorflow.org/api_docs/python/tf/keras/metrics).

In [ ]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## Training the model on federated data

Now that we have a model wrapped as `tff.learning.Model` for use with TFF, we
can let TFF construct a Federated Averaging algorithm by invoking the helper
function `tff.learning.build_federated_averaging_process`, as follows.

In [ ]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    # Add server optimizer here!
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))
    

What just happened? TFF has constructed a pair of *federated computations* and packaged them into a `tff.templates.IterativeProcess` in which these computations are available as a pair of properties `initialize` and `next`.

반복 프로세스는 일반적으로 다음과 같은 제어 루프에 의해 구동됩니다.

```
def initialize():
  ...

def next(state):
  ...

iterative_process = IterativeProcess(initialize, next)
state = iterative_process.initialize()
for round in range(num_rounds):
  state = iterative_process.next(state)
```


Let's invoke the `initialize` computation to construct the server state.

In [ ]:
state = iterative_process.initialize()

The second of the pair of federated computations, `next`, represents a single
round of Federated Averaging, which consists of pushing the server state
(including the model parameters) to the clients, on-device training on their
local data, collecting and averaging model updates, and producing a new updated
model at the server.

Let's run a single round of training and visualize the results. We can use the
federated data we've already generated above for a sample of users.

In [ ]:
# Run one single round of training.
state, metrics = iterative_process.next(state, federated_train_data)
print('round  1, metrics={}'.format(metrics['train']))

Let's run a few more rounds. As noted earlier, typically at this point you would
pick a subset of your simulation data from a new randomly selected sample of
users for each round in order to simulate a realistic deployment in which users
continuously come and go, but in this interactive notebook, for the sake of
demonstration we'll just reuse the same users, so that the system converges
quickly.

In [ ]:
NUM_ROUNDS = 11
for round_num in range(2, NUM_ROUNDS):
  state, metrics = iterative_process.next(state, federated_train_data)
  print('round {:2d}, metrics={}'.format(round_num, metrics['train']))

Training loss is decreasing after each round of federated training, indicating
the model is converging. There are some important caveats with these training
metrics, however, see the section on *Evaluation* later in this tutorial.

##Displaying model metrics in TensorBoard
Next, let's visualize the metrics from these federated computations using Tensorboard.

Let's start by creating the directory and the corresponding summary writer to write the metrics to.


In [ ]:
#@test {"skip": true}
import os
import shutil

logdir = "/tmp/logs/scalars/training/"
if os.path.exists(logdir):
  shutil.rmtree(logdir)

# Your code to create a summary writer:
summary_writer = tf.summary.create_file_writer(logdir)

state = iterative_process.initialize()

Plot the relevant scalar metrics with the same summary writer.

In [ ]:
#@test {"skip": true}
with summary_writer.as_default():
  for round_num in range(1, NUM_ROUNDS):
    state, metrics = iterative_process.next(state, federated_train_data)
    for name, value in metrics['train'].items():
      tf.summary.scalar(name, value, step=round_num)

Start TensorBoard with the root log directory specified above. It can take a few seconds for the data to load.

In [ ]:
#@test {"skip": true}
%tensorboard --logdir /tmp/logs/scalars/ --port=0

In order to view evaluation metrics the same way, you can create a separate eval folder, like "logs/scalars/eval", to write to TensorBoard.

## 평가

To perform evaluation on federated data, you can construct another *federated computation* designed for just this purpose, using the `tff.learning.build_federated_evaluation` function, and passing in your model constructor as an argument.

In [ ]:
# Construct federated evaluation computation here:
evaluation = tff.learning.build_federated_evaluation(model_fn)

Now, let's compile a test sample of federated data and rerun evaluation on the
test data. The data will come from a different sample of users, but from a
distinct held-out data set.

In [ ]:
import random
shuffled_ids = emnist_test.client_ids.copy()
random.shuffle(shuffled_ids)
sample_clients = shuffled_ids[0:NUM_CLIENTS]

federated_test_data = make_federated_data(emnist_test, sample_clients)

len(federated_test_data), federated_test_data[0]

In [ ]:
# Run evaluation on the test data here, using the federated model produced from 
# training:
test_metrics = evaluation(state.model, federated_test_data)

In [ ]:
str(test_metrics)

This concludes the tutorial. We encourage you to play with the
parameters (e.g., batch sizes, number of users, epochs, learning rates, etc.), to modify the code above to simulate training on random samples of users in
each round, and to explore the other tutorials we've developed.

# Build your own FL algorithms

In the previous tutorials, we learned how to set up model and data pipelines, and use these to perform federated training using the `tff.learning` API.

Of course, this is only the tip of the iceberg when it comes to FL research. In this tutorial, we are going to discuss how to implement federated learning algorithms *without* deferring to the `tff.learning` API. We aim to accomplish the following:

**Goals:**

- Understand the general structure of federated learning algorithms.
- Explore the *Federated Core* of TFF.
- Federated Core를 사용하여 Federated Averaging을 직접 구현합니다.


## 입력 데이터 준비

We first load and preprocess the EMNIST dataset included in TFF. We essentially use the same code as in the first tutorial.

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

In [ ]:
client_ids = np.random.choice(emnist_train.client_ids, size=NUM_CLIENTS, replace=False)

federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## Preparing the model

We use the same model as the first tutorial, which has a single hidden layer, followed by a softmax layer.

In [ ]:
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])

We wrap this Keras model as a `tff.learning.Model`.

In [ ]:
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Cutomizing FL Algorithm

While the `tff.learning` API encompasses many variants of Federated Averaging, there are many other algorithms that do not fit neatly into this framework. For example, you may want to add regularization, clipping, or more complicated algorithms such as [federated GAN training](https://github.com/google-research/federated/blob/master/gans). You may also be instead be interested in [federated analytics](https://ai.googleblog.com/2020/05/federated-analytics-collaborative-data.html).

이러한 고급 알고리즘의 경우 사용자 정의 FL 알고리즘을 작성해야합니다.

일반적으로 FL 알고리즘에는 4 가지 주요 구성 요소가 있습니다.

1. 서버-클라이언트 브로드 캐스트 단계.
2. 로컬 클라이언트 업데이트 단계.
3. 클라이언트-서버 업로드 단계.
4. 서버 업데이트 단계.

TFF에서 우리는 일반적으로 연합 알고리즘을 `IterativeProcess` 로 표현합니다. 이것은 `initialize_fn` 및 `next_fn` 을 포함하는 단순한 클래스입니다. `initialize_fn` 서버 초기화하는 데 사용되며, 상기 `next_fn` 연합 평균화 한 라운드 통신을 수행한다. FedAvg에 대한 우리의 반복 프로세스가 어떻게 생겼는지에 대한 골격을 작성해 보겠습니다.

먼저 `tff.learning.Model` 생성하고 훈련 가능한 가중치를 반환하는 초기화 함수가 있습니다.

In [ ]:
def initialize_fn():
  model = model_fn()
  return model.weights.trainable

이 함수는 괜찮아 보이지만 나중에 살펴 보 겠지만 TFF 계산을 위해 약간의 수정이 필요합니다.

We also want to sketch the `next_fn`.

In [ ]:
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = client_update(federated_dataset, server_weights_at_client)

  # The server averages these updates.
  mean_client_weights = mean(client_weights)

  # The server updates its model.
  server_weights = server_update(mean_client_weights)

  return server_weights

이 네 가지 구성 요소를 개별적으로 구현하는 데 중점을 둘 것입니다. 먼저 순수 TensorFlow에서 구현할 수있는 부분, 즉 클라이언트 및 서버 업데이트 단계에 초점을 맞출 것입니다.


## TensorFlow 블록 

### 클라이언트 업데이트

`tff.learning.Model` 을 사용하여 기본적으로 TF 모델을 학습하는 것과 동일한 방식으로 클라이언트 학습을 수행합니다. 특히 `tf.GradientTape` 를 사용하여 데이터 배치에 대한 그라디언트를 계산 한 다음 `client_optimizer` 사용하여 이러한 그라디언트를 적용합니다.

각 `tff.learning.Model` 인스턴스에는 두 개의 하위 속성이있는 `weights` 속성이 있습니다.

- `trainable` : 훈련 가능한 레이어에 해당하는 텐서 목록입니다.
- `non_trainable` : 훈련 불가능한 레이어에 해당하는 텐서 목록입니다.

우리의 목적을 위해 우리는 훈련 가능한 가중치만을 사용할 것입니다 (우리 모델에는 그것들 만 있습니다!).


In [ ]:
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.weights.trainable
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

### 서버 업데이트

서버 업데이트에는 더 적은 노력이 필요합니다. 우리는 단순히 서버 모델 가중치를 클라이언트 모델 가중치의 평균으로 대체하는 바닐라 연합 평균을 구현할 것입니다. 다시 말하지만, 훈련 가능한 가중치에만 초점을 맞출 것입니다.

In [ ]:
@tf.function
def server_update(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.weights.trainable
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

위의 코드 스 니펫은 단순히 `mean_client_weights` 를 반환 할 수 있으므로 분명히 과잉입니다. 그러나 Federated Averaging의 고급 구현에서는 모멘텀 또는 적응성과 같은보다 정교한 기술과 함께 `mean_client_weights` 를 사용할 수 있습니다.

지금까지 순수한 TensorFlow 코드 만 작성했습니다. TFF를 사용하면 이미 익숙한 TensorFlow 코드를 많이 사용할 수 있으므로 이는 의도적으로 설계된 것입니다. 그러나 이제 *오케스트레이션 로직* , 즉 서버가 클라이언트에 브로드 캐스트하는 내용과 클라이언트가 서버에 업로드하는 내용을 지정하는 로직을 지정해야합니다.

이를 위해서는 TFF의 "Federated Core"가 필요합니다.

# 연합 코어 소개

연합 코어 (FC)는 `tff.learning` API의 기반 역할을하는 하위 수준 인터페이스 집합입니다. 그러나 이러한 인터페이스는 학습에만 국한되지 않습니다. 실제로 분산 데이터에 대한 분석 및 기타 많은 계산에 사용할 수 있습니다.

상위 수준에서 연합 코어는 간결하게 표현 된 프로그램 로직을 사용하여 TensorFlow 코드를 분산 통신 연산자 (예 : 분산 합계 및 브로드 캐스트)와 결합 할 수있는 개발 환경입니다. 목표는 시스템 구현 세부 사항 (예 : 지점 간 네트워크 메시지 교환 지정)을 요구하지 않고 연구자와 실무자에게 시스템의 분산 통신을 신속하게 제어 할 수 있도록하는 것입니다.

한 가지 요점은 TFF가 개인 정보 보호를 위해 설계되었다는 것입니다. 따라서 중앙 집중식 서버 위치에서 원치 않는 데이터 축적을 방지하기 위해 데이터가있는 위치를 명시 적으로 제어 할 수 있습니다.

## 연합 데이터

기본 개념 중 하나 인 TensorFlow의 "Tensor"개념과 유사하게 TFF의 핵심 개념은 분산 시스템의 장치 그룹 (예 : 클라이언트 데이터 세트 또는 서버 모델 가중치). 우리는 모든 장치에 걸친 전체 데이터 항목 컬렉션을 하나의 *연합 값* 으로 모델링합니다.

예를 들어, 센서의 온도를 나타내는 부동 소수점이있는 클라이언트 장치가 있다고 가정합니다. 우리는 그것을 *연합 된 float* 로 표현할 수 있습니다.

In [ ]:
federated_float_on_clients = tff.type_at_clients(tf.float32)

Federated types are specified by a type `T` of its member constituents (eg. `tf.float32`) and a group `G` of devices. We will focus on the cases where `G` is either `tff.CLIENTS` or `tff.SERVER`. Such a federated type is represented as `{T}@G`, as shown below.

In [ ]:
str(federated_float_on_clients)

게재 위치에 관심이 많은 이유는 무엇입니까? TFF의 핵심 목표는 실제 분산 시스템에 배포 할 수있는 코드를 작성할 수 있도록하는 것입니다. 즉, 장치의 하위 집합이 어떤 코드를 실행하고 다른 데이터 조각이 어디에 있는지 추론하는 것이 중요합니다.

TFF는 *데이터* , 데이터가 *배치되는 위치* 및 데이터가 *변환* 되는 방법의 세 가지에 중점을 둡니다. 처음 두 개는 연합 유형으로 캡슐화되고 마지막 두 개는 *연합 계산에* 캡슐화됩니다.

## 통합 계산

TFF는 기본 단위가 *연합 계산* 인 강력한 형식의 함수형 프로그래밍 환경입니다. 이들은 연합 값을 입력으로 받아들이고 연합 값을 출력으로 반환하는 논리 조각입니다.

예를 들어 클라이언트 센서의 온도를 평균화하고 싶다고 가정합니다. 다음을 정의 할 수 있습니다 (페더레이션 된 float 사용).

In [ ]:
@tff.federated_computation(tff.type_at_clients(tf.float32))
def get_average_temperature(client_temperatures):
  return tff.federated_mean(client_temperatures)

TensorFlow의 `tf.function` 데코레이터와 어떻게 다른지 물어볼 수 있습니다. 핵심 대답은 `tff.federated_computation` 에 의해 생성 된 코드가 TensorFlow도 아니고 Python 코드도 아니라는 것입니다. 내부 플랫폼 독립적 인 *글루 언어로* 된 분산 시스템의 사양입니다.

복잡하게 들릴 수 있지만 TFF 계산은 잘 정의 된 형식 서명이있는 함수로 생각할 수 있습니다. 이러한 유형 서명은 직접 쿼리 할 수 있습니다.

In [ ]:
str(get_average_temperature.type_signature)

이 `tff.federated_computation` 연합 형의 인수를 받아들이는 `{float32}@CLIENTS` 및 연합 유형의 반환 값 `{float32}@SERVER` . 연합 계산은 서버에서 클라이언트로, 클라이언트에서 클라이언트로 또는 서버에서 서버로 이동할 수도 있습니다. 연합 계산은 유형 서명이 일치하는 한 일반 함수처럼 구성 할 수도 있습니다.

개발을 지원하기 위해 TFF를 사용하면 `tff.federated_computation` 을 Python 함수로 호출 할 수 있습니다. 예를 들어

In [ ]:
get_average_temperature([68.5, 70.3, 69.8])

## 열망하지 않는 계산 및 TensorFlow

주의해야 할 두 가지 주요 제한 사항이 있습니다. 첫째, 파이썬 인터프리터가 `tff.federated_computation` 데코레이터를 만나면 함수가 한 번 추적되고 나중에 사용하기 위해 직렬화됩니다. 따라서 TFF 계산은 근본적으로 *비열 심적* 입니다. 이 동작은 TensorFlow의 [`tf.function`](https://www.tensorflow.org/api_docs/python/tf/function) 데코레이터와 다소 유사합니다.

둘째, 연합 계산은 연합 연산자 (예 : `tff.federated_mean` )로만 구성 될 수 있으며 TensorFlow 작업을 포함 할 수 없습니다. TensorFlow 코드는 `tff.tf_computation` 장식 된 블록으로 제한되어야합니다. 숫자를 취하고 여기에 `0.5` 를 더하는 다음 함수와 같이 대부분의 일반적인 TensorFlow 코드는 직접 decotrated 할 수 있습니다.

In [ ]:
@tff.tf_computation(tf.float32)
def add_half(x):
  return tf.add(x, 0.5)

또한 유형 서명이 있지만 *배치는 없습니다* . 예를 들어

In [ ]:
str(add_half.type_signature)

여기에서 `tff.federated_computation` 과 `tff.tf_computation` 의 중요한 차이점을 볼 수 있습니다. 전자는 명시 적 배치가있는 반면 후자는 그렇지 않습니다.

We can use `tff.tf_computation` blocks in federated computations by specifying placements. Let's create a function that adds half, but only to federated floats at the clients. We can do this by using `tff.federated_map`, which applies a given `tff.tf_computation`, while preserving the placement.

In [ ]:
@tff.federated_computation(tff.type_at_clients(tf.float32))
def add_half_on_clients(x):
  return tff.federated_map(add_half, x)

이 기능은 거의 동일 `add_half` , 그것은 단지에서 위치와 값을 받아들이는 것을 제외하고 `tff.CLIENTS` 같은 배치로, 반환 값을 설정합니다. 유형 서명에서 이것을 볼 수 있습니다.

In [ ]:
str(add_half_on_clients.type_signature)

요약하자면:

- TFF는 연합 값에서 작동합니다.
- 각 연합 값은 *유형* (예.와 *연합 형* 갖는다 `tf.float32` ) 및 *위치* (예. `tff.CLIENTS` ).
- 연합 값은 *연합 계산을* 사용하여 변환 될 수 있으며, `tff.federated_computation` 및 연합 유형 서명으로 장식되어야합니다.
- TensorFlow 코드는 `tff.tf_computation` 데코레이터가있는 블록에 포함되어야합니다.
- 그런 다음 이러한 블록을 통합 계산에 통합 할 수 있습니다.


# 나만의 FL 알고리즘 구축 (2 부)

이제 연합 코어를 살펴 보았으므로 자체 연합 학습 알고리즘을 구축 할 수 있습니다. 위에서 알고리즘에 `initialize_fn` 및 `next_fn` 을 정의했습니다. `next_fn` 은 순수한 TensorFlow 코드를 사용하여 정의한 `client_update` 및 `server_update` 를 사용합니다.

그러나 알고리즘을 연합 계산으로 만들기 위해서는 `next_fn` 과 `initialize_fn` 이 모두 `tff.federated_computations` 합니다.

## TensorFlow 연합 블록 

### 초기화 계산 만들기

initialize 함수는 매우 간단합니다. `model_fn` 사용하여 모델을 생성합니다. 그러나 `tff.tf_computation` 사용하여 TensorFlow 코드를 분리해야합니다.

In [ ]:
@tff.tf_computation
def server_init():
  model = model_fn()
  return model.weights.trainable

We can then pass this directly into a federated computation using `tff.federated_value`.

In [ ]:
@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(), tff.SERVER)

### `next_fn` 만들기

이제 클라이언트 및 서버 업데이트 코드를 사용하여 실제 알고리즘을 작성합니다. 우리는 먼저 우리의 켜집니다 `client_update` 에 `tff.tf_computation` 클라이언트 데이터 셋 및 서버 가중치를 받아들이고, 업데이트 된 클라이언트 무게 텐서를 출력합니다.

함수를 적절하게 꾸미려면 해당 유형이 필요합니다. 다행히도 서버 가중치 유형은 모델에서 직접 추출 할 수 있습니다.

In [ ]:
whimsy_model = model_fn()
tf_dataset_type = tff.SequenceType(whimsy_model.input_spec)

데이터 세트 유형 서명을 살펴 보겠습니다. 28 x 28 이미지 (정수 레이블 포함)를 가져와 병합했다는 것을 기억하십시오.

In [ ]:
str(tf_dataset_type)

위의 `server_init` 함수를 사용하여 모델 가중치 유형을 추출 할 수도 있습니다.

In [ ]:
model_weights_type = server_init.type_signature.result

형식 서명을 살펴보면 모델의 아키텍처를 볼 수 있습니다!

In [ ]:
str(model_weights_type)

이제 클라이언트 업데이트를위한 `tff.tf_computation` 을 생성 할 수 있습니다.

In [ ]:
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update(model, tf_dataset, server_weights, client_optimizer)

서버 업데이트의 `tff.tf_computation` 버전은 이미 추출한 유형을 사용하여 유사한 방식으로 정의 할 수 있습니다.

In [ ]:
@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)

Last, but not least, we need to create the `tff.federated_computation` that brings this all together. This function will accept two *federated values*, one corresponding to the server weights (with placement `tff.SERVER`), and the other corresponding to the client datasets (with placement `tff.CLIENTS`).

Note that both these types were defined above! We simply need to give them the proper placement using `tff.type_at_{server/clients}``.

In [ ]:
federated_server_type = tff.type_at_server(model_weights_type)
federated_dataset_type = tff.type_at_clients(tf_dataset_type)

FL 알고리즘의 4 가지 요소를 기억하십니까?

1. 서버-클라이언트 브로드 캐스트 단계.
2. 로컬 클라이언트 업데이트 단계.
3. 클라이언트-서버 업로드 단계.
4. 서버 업데이트 단계.

이제 위의 내용을 작성 했으므로 각 부분을 한 줄의 TFF 코드로 간결하게 표현할 수 있습니다. 이 단순함 때문에 연합 유형과 같은 것을 지정하기 위해 특별히주의해야했습니다!

In [ ]:
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(
      client_update_fn, (federated_dataset, server_weights_at_client))
  
  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights

We now have a `tff.federated_computation` for both the algorithm initialization, and for running one step of the algorithm. To finish our algorithm, we pass these into `tff.templates.IterativeProcess`.

In [ ]:
federated_algorithm = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn,
    next_fn=next_fn
)

반복 프로세스의 `initialize` 및 `next` 함수의 *유형 시그니처* 를 살펴 보겠습니다.

In [ ]:
str(federated_algorithm.initialize.type_signature)

이는 `federated_algorithm.initialize` 가 단일 레이어 모델 (784x10 가중치 행렬 및 10 개의 바이어스 단위 포함)을 반환하는 인수가없는 함수라는 사실을 반영합니다.

In [ ]:
str(federated_algorithm.next.type_signature)

여기에서 `federated_algorithm.next` 는 서버 모델과 클라이언트 데이터를 받아들이고 업데이트 된 서버 모델을 반환합니다.

## 알고리즘 평가

몇 라운드를 실행하고 손실이 어떻게 변하는 지 살펴 보겠습니다. 먼저 두 번째 튜토리얼에서 논의한 *중앙 집중식* 접근 방식을 사용하여 평가 기능을 정의합니다.

먼저 중앙 집중식 평가 데이터 세트를 만든 다음 훈련 데이터에 사용한 것과 동일한 전처리를 적용합니다.

주 우리는 것을 `take` 계산 효율성을 이유로 처음 1000 개 요소가 있지만, 일반적으로 우리는 전체 테스트 데이터 세트를 사용하십시오.

In [ ]:
central_emnist_test = emnist_test.create_tf_dataset_from_all_clients().take(1000)
central_emnist_test = preprocess(central_emnist_test)

다음으로 서버 상태를 받아들이고 Keras를 사용하여 테스트 데이터 세트를 평가하는 함수를 작성합니다. 당신이 익숙하다면 `tf.Keras` ,이 모든 모습을 잘 알고, 참고하지만 사용 `set_weights` !

In [ ]:
def evaluate(server_state):
  keras_model = create_keras_model()
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]  
  )
  keras_model.set_weights(server_state)
  keras_model.evaluate(central_emnist_test)

이제 알고리즘을 초기화하고 테스트 세트에서 평가 해 보겠습니다.

In [ ]:
server_state = federated_algorithm.initialize()
evaluate(server_state)

몇 라운드 동안 훈련하고 변경 사항이 있는지 살펴 보겠습니다.

In [ ]:
for round in range(15):
  server_state = federated_algorithm.next(server_state, federated_train_data)

In [ ]:
evaluate(server_state)

손실 함수가 약간 감소합니다. 점프는 작지만, 우리는 10 번의 훈련 라운드와 소수의 클라이언트에 대해서만 수행했습니다. 더 나은 결과를 보려면 수천 번은 아니더라도 수백 번을해야 할 수도 있습니다.

## 알고리즘 수정

이 시점에서 멈추고 우리가 성취 한 것에 대해 생각해 봅시다. 순수 TensorFlow 코드 (클라이언트 및 서버 업데이트 용)를 TFF의 Federated Core의 연합 계산과 결합하여 Federated Averaging을 직접 구현했습니다.

보다 정교한 학습을 수행하려면 위에있는 내용을 간단히 변경할 수 있습니다. 특히 위의 순수 TF 코드를 편집하여 클라이언트가 학습을 수행하는 방법 또는 서버가 모델을 업데이트하는 방법을 변경할 수 있습니다.

**Challenge:** Add [gradient clipping](https://towardsdatascience.com/what-is-gradient-clipping-b8e815cdfb48) to the `client_update` function.
